# web scraping RIP.ie with beautifulsoup

In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    if soup.title == None:
        title = ''
    else:        
        title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [2]:
get_dn_page(390045)

390045 2019-06-05 00:00:00 Margaret  Kelly Avenue Grove, Ballymodan Place, Bandon,  Cork Cork


('Margaret  Kelly',
 datetime.datetime(2019, 6, 5, 0, 0),
 'Cork',
 'Avenue Grove, Ballymodan Place, Bandon,  Cork',
 "Kelly (Avenue Grove, Ballymodan Place, Bandon and late of Currivreeda West) on June 5th 2019. Margaret, beloved daughter of the late Timothy and Ellen. Sadly missed by her loving sisters Ann and Eileen, brothers Teddy and John, Margaret's Partner William, brothers-in-law, sisters-in-law, nieces, nephews, relatives and good friend Malcolm.;;Rosary on Friday evening at 7pm in St. Patrick's Church, Bandon. Requiem Mass on Saturday at 12 noon, funeral afterwards to the adjoining cemetery.;;May Margaret Rest in Peace")

In [28]:
#df = pd.read_csv('rip_dn_scrape.csv')
df = pd.read_pickle('rip_dn_scrape.pkl')
len(df)

473362

In [34]:
z.to_csv('/home/damien/Downloads/test_day.csv')

## iterate over a range of ids and get info

In [2]:
#read current table in so we skip those already done
df = pd.read_pickle('rip_dn_scrape.pkl')
print (len(df))
ids = list(df.id)

results={}
for n in range(478000,482066):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page(n)
    if name == '':
        continue
    results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

482064 2022-01-06 00:00:00 Jack  MURPHY Aghabullogue,  Cork Cork
482065 2022-01-05 00:00:00 Mary  DINAN Bray,  Wicklow Wicklow


In [3]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()
res = res.rename(columns={'index':'id'})

<ipython-input-3-849ec239fce7>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()


In [4]:
res

,id,name,date,county,address,notice
0,481698,Donal (Bazil) Shiels,2022-01-02,Donegal,"Carrickataskin, Derrybeg, Donegal","The death has taken place, at Letterkenny Univ..."
1,481699,Catherine (Kate) FARRELL (née Matthews),2022-01-03,Meath,"Beaubec, Dublin Raod, Drogheda, Louth",FARRELL Catherine (Kate) (née Matthews) (Beaub...
2,481700,Richard SINNOTT,2022-01-03,Dublin,"Sandymount, Dublin","SINNOTT – Richard (Sandymount, Dublin; formerl..."
3,481701,Sean McGoldrick,2022-01-03,Donegal,"Ard O'Donnell, Letterkenny, Donegal","The death has occurred, at the Donegal Hospice..."
4,481702,Sr. Angela Tansey,2022-01-04,Mayo,"Knock, Mayo / Ballinameen, Roscommon","Sr. Angela Tansey, Holy Family Sister, Knock C..."
...,...,...,...,...,...,...
363,482061,Joe GUEST,2022-01-05,Louth,"Clann Chullainn Park, Dundalk, Louth / Newtow...","Late of Slate Quarry Road, Newtownhamilton. Pr..."
364,482062,Margaret (Mairead) O'Halloran (née Daly),2022-01-06,Tipperary,"Nenagh, Tipperary / Ogonnelloe, Clare","Margaret (Mairead) O'Halloran, (nee Daly), Cur..."
365,482063,William Watson,2022-01-06,Dublin,"late of Dunard Ave., Dublin 7, Dublin","Watson, William 6th January 2022, late of Duna..."
366,482064,Jack MURPHY,2022-01-06,Cork,"Aghabullogue, Cork","On January 6th, 2022, peacefully at Grangecon ..."


In [5]:
x = pd.concat([df,res]).reset_index(drop=True)
x=x[~x.id.duplicated(keep='first')]
print (len(df),len(res),len(x))
x.to_pickle('rip_dn_scrape.pkl')

481670 368 482038


In [6]:
print (len(x))
x=x.replace('',None).dropna(axis=0,subset=['date'])
x['date'] = pd.to_datetime(x['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
x=x.drop_duplicates(['name','address'])
x=x.drop_duplicates(['name','date','county'])
x=x.sort_values('id')
print (len(x))
#x.to_csv('rip_dn_scrape_processed.csv')
x.to_pickle('rip_dn_scrape_processed.pkl')

482038
462131
